# Structural Estimation
# PS4
# Jose Cerda

# Estimating the Brock and Mirman (1972) model by SMM

In [ ]:
#Check
#Adjust number of simulation from 10 to 1000 (when ready)
#Corr in data moments
#More than 1 dimension in data moments for simulations

In [126]:
# Import the necessary libraries
import numpy.linalg as lin
import scipy.integrate as intgr
import numpy as np
import scipy.stats as sts
import requests
import statistics
import matplotlib.pyplot as plt
import pandas as pd
import scipy.special as spc
import os
import random
import scipy.optimize as opt
from pandas import DataFrame
from pandas import concat
import seaborn as sns
sns.set()

In [127]:
#Import data
macroseries = np.loadtxt('data/NewMacroSeries.txt', delimiter=',')

In [128]:
#Reshape data
c = macroseries[:,[0]]
k = macroseries[:,[1]]
w = macroseries[:,[2]]
r = macroseries[:,[3]]
y = macroseries[:,[4]]
print('c4',c[4],'k4',k[4],'w4',w[4],'r4',r[4], 'y4', y[4])
#df = pd.DataFrame(macroseries)
#df.columns = ['c', 'k', 'w', 'r', 'y']

c4 [9245673.48747698] k4 [6912184.01188691] w4 [9179203.3939347] r4 [0.96163663] y4 [15826212.74816327]


In [129]:
#Initial parameters
alpha_0=0.3
beta=0.99
rho_0=0.4
mu_0=0.7
sigma_0=0.6
np.random.seed(100)

#Parameters
alpha=alpha_0
#beta=beta_0
rho=rho_0
mu=mu_0
sigma=sigma_0

## First, assume that z0 = mu and that k1 = mean(kt) from the data. Also assume that beta = 0.99. Next, draw a matrix of S = 1000 simulations (columns) of T = 100 (rows) from a uniform distribution

In [130]:
# Assumptions
z_0 = mu
k_1 = k.mean()
print('z_0 =', z_0, 'k_1 =', k_1)

z_0 = 0.7 k_1 = 6643985.138299068


In [131]:
# Define LN dist
def ln_norm_pdf(xvals, mu, sigma):
    pdf_vals = ((1/(xvals * sigma * np.sqrt(2 * np.pi))) * np.exp( - (np.log(xvals) - mu)**2 / (2 * sigma**2)))
    pdf_vals[pdf_vals < 1e-10] = 1e-10
    return pdf_vals

In [132]:
# Define function that draws N x S values
def norm_draws(unif_vals, mu, sigma):
    norm_draws_vals = sts.norm.ppf(unif_vals, loc=mu, scale=sigma)
    return norm_draws_vals

In [133]:
# Priors to simulations
#Parameters of simulations
alpha_s = 0.5
beta = 0.99
mu_s = 10
sigma_s = 0.1
rho_s = 0.7

In [134]:
#Simulations function
def sim_fn(alpha_s, mu_s, sigma_s, rho_s):
    
    t=100 #Size of the observation matrix to simulate
    s=1000 #Number of simulations (Change to 1000 when ready)
    beta = 0.99
    err_mu = 0
    err_sigma = sigma_s
    np.random.seed(100) # Set the random number seed so it gives same answers every time
    unif_vals = sts.uniform.rvs(0, 1, size=(t, s))
    
    #Errors simulation
    err_s = norm_draws(unif_vals, 0, err_sigma)

    #z simulation
    #Compute z and z[t-1]
    z_s = np.zeros((t, s))
    zlag_s = np.zeros((t, s))
    z_s[0,:] = mu_s+err_s[0,:]
    for i in range(1, t):
        for j in range(0,s):
            zlag_s[i,j] = z_s[i-1,j]
            z_s[i,j] = rho_s*(zlag_s[i,j])+(1-rho_s)*mu_s+err_s[i,j]
        
    #k simulation
    k_s = np.zeros((t, s))
    k_s[0,:] = k.mean()
    for i in range(0,t-1):
        for j in range(0, s):
            k_s[i+1,j] = alpha_s*beta*np.exp(z_s[i,j])*k_s[i,j]**alpha_s
        
    #w simulation
    w_s = np.zeros((t, s))
    for i in range(0, t):
        for j in range(0,s):
            w_s[i,j] = (1-alpha_s)*(np.exp(z_s[i,j]))*((k_s[i,j])**alpha_s)
        
    #r simulation
    r_s = np.zeros((t, s))
    for i in range(0, t):
        for j in range(0,s):
            r_s[i,j] = alpha_s*np.exp(z_s[i,j])*(k_s[i,j])**(alpha_s-1)
        
    #c simulation
    c_s = np.zeros((t, s))
    for i in range(0, t-1):
        for j in range(0,s):
            c_s[i,j] = w_s[i,j]+(r_s[i,j])*(k_s[i,j])-k_s[i+1,j]
        
    #y simulation
    y_s = np.zeros((t, s))
    for i in range(0, t):
        for j in range(0,s):
            y_s[i,j] = np.exp(z_s[i,j])*k_s[i,j]**alpha_s
        
    #Compute consumption share
    c_sh_s = np.zeros((t, s))
    for i in range(0, t):
        for j in range(0,s):
            c_sh_s[i,j] = c_s[i,j]/y_s[i,j]
        
    #Compute consumption lag
    #c_lag_s = np.zeros((t, s))
    c_lag_s = np.ones((t, s))
    #c_lag_s = c_s
    for i in range(0, t):
        for j in range(0,s):
            c_lag_s[i,j] = c_s[i-1,j]
        
    return c_s, k_s, w_s, r_s, y_s, c_sh_s, c_lag_s
    

# a) Estimate four parameters. Choose the four parameters to match the following six moments: mean(ct), mean(kt), mean(ct/yt), var(yt), corr(ct, ct-1), corr(ct; kt)

In [135]:
#Newvars
#Consumption share
c_sh = c/y
#Compute c[t-1]
c_df = DataFrame(c)
c_df_mat = concat([c_df.shift(-1), c_df], axis=1)
c_df_mat.columns = ['t-1', 't+1']
#c_lag = c_df_mat.shift(1)

In [136]:
#Define the data moments
def moments_fn(c, k, w, r, y, c_sh, c_lag):
        
    #calc c_lag
    c_lag = pd.DataFrame(c).shift(1)
    c_lag_arr = np.asarray(c)
    #c_lag_arr = np.asarray(c_lag)
    
    #Moments
    if c.ndim == 1:
        m1_fn = c.mean()
        m2_fn = k.mean()
        m3_fn = c_sh.mean()
        m4_fn = y.var()
        m5_fn = sts.pearsonr(c,c_lag_arr)[0]
        m6_fn = sts.pearsonr(c,k)[0]
    elif c.ndim == 2:
        m1_fn = statistics.mean(c.mean(axis=0))
        m2_fn = statistics.mean(k.mean(axis=0))
        m3_fn = statistics.mean(c_sh.mean(axis=0))
        m4_fn = statistics.mean(y.var(axis=0))
        c_avg = c.mean(axis=1)
        k_avg = k.mean(axis=1)
        c_lag_avg = c_lag_arr.mean(axis=1)
        m5_fn = sts.pearsonr(c_avg, c_lag_avg)[0]
        m6_fn = sts.pearsonr(c_avg, k_avg)[0]

    return m1_fn, m2_fn, m3_fn, m4_fn, m5_fn, m6_fn   

In [137]:
#Compute the data moments
m1_dta, m2_dta, m3_dta, m4_dta, m5_dta, m6_dta = moments_fn(c, k, w, r, y, c_sh, c_lag)
dta_moms = np.array([[m1_dta], [m2_dta], [m3_dta], [m4_dta], [m5_dta], [m6_dta]])
print('m1_dta', m1_dta)
print('m2_dta', m2_dta)
print('m3_dta', m3_dta)
print('m4_dta', m4_dta)
print('m5_dta', m5_dta)
print('m6_dta', m6_dta)

m1_dta 9281790.485669706
m2_dta 6643985.138299068
m3_dta 0.5842000000000002
m4_dta 28377825058899.727
m5_dta 0.9999999999999999
m6_dta 0.9408030537975818


In [138]:
#Compute the model moments prior
c_sim1, k_sim1, w_sim1, r_sim1, y_sim1, c_sh_sim1, c_lag_sim1 = sim_fn(alpha_s, mu_s, sigma_s, rho_s)
m1_mom, m2_mom, m3_mom, m4_mom, m5_mom, m6_mom = moments_fn(c_sim1, k_sim1, w_sim1, r_sim1, y_sim1, c_sh_sim1, c_lag_sim1)
mod_moms = np.array([[m1_mom], [m2_mom], [m3_mom], [m4_mom], [m5_mom], [m6_mom]])
print('m1_mom', m1_mom)
print('m2_mom', m2_mom)
print('m3_mom', m3_mom)
print('m4_mom', m4_mom)
print('m5_mom', m5_mom)
print('m6_mom', m6_mom)

m1_mom 120884352.63416785
m2_mom 118557042.9284386
m3_mom 0.49995000000000067
m4_mom 3594274374123498.5
m5_mom 1.0
m6_mom 0.6900841196225584


In [140]:
#print(c_sim1, c_lag_sim1)

In [141]:
#Error vector
def err_vec(sim_vals, simple):
    c_sim, k_sim, w_sim, r_sim, y_sim, c_sh_sim, c_lag_sim = sim_vals
    
    #Data moments
    m1_dta, m2_dta, m3_dta, m4_dta, m5_dta, m6_dta = moments_fn(c, k, w, r, y, c_sh, c_lag)
    moms_data = np.array([[m1_dta], [m2_dta], [m3_dta], [m4_dta], [m5_dta], [m6_dta]])
    
    #Model moments
    m1_mom, m2_mom, m3_mom, m4_mom, m5_mom, m6_mom = moments_fn(c_sim, k_sim, w_sim, r_sim, y_sim, c_sh_sim, c_lag_sim)
    moms_sim = np.array([[m1_mom], [m2_mom], [m3_mom], [m4_mom], [m5_mom], [m6_mom]])

    #Err vec
    if simple:
        err_vec = moms_sim - moms_data
    else:
        err_vec = (moms_sim - moms_data) / moms_data
    
    return err_vec
    
    

In [142]:
#Criterion function
def criterion(params, *args):
    alpha, mu, sigma, rho = params
    sim_vals, W_hat = args
    sim_vals = sim_fn(alpha, mu, sigma, rho)
    err = err_vec(sim_vals, simple=True)
    crit_val = err.T @ W_hat @ err 
    return crit_val    
    

In [143]:
#Estimate by SMM
#Initial parameters
alpha_init = alpha_s
beta = 0.99
mu_init = mu_s
sigma_init = sigma_s
rho_init = rho_s
params_init = np.array([alpha_init, mu_init, sigma_init, rho_init])
W_hat = np.eye(6)
sim_init = sim_fn(alpha_init, mu_init, sigma_init, rho_init)

smm_args = (sim_init, W_hat)

results = opt.minimize(criterion, params_init, args=(smm_args), method='TNC', bounds=((0.01-(1e-10), 0.99+(1e-10)), (5-(1e-10), 14+(1e-10)), (0.01-(1e-10), 1.1+(1e-10)), (-0.99-(1e-10), 0.99+(1e-10))))

#results = opt.minimize(criterion, params_init, args=(smm_args), method='L-BFGS-B', bounds=((1e-10, 1-(1e-10)), (5+(1e-10), 14-(1e-10)), (1e-10, 1.1-(1e-10)), (-1+(1e-10), 1-(1e-10))))
alpha_smm, mu_smm, sigma_smm, rho_smm = results.x
print('alpha_smm=', alpha_smm, 'mu_smm=', mu_smm, 'sigma_smm=', sigma_smm, 'rho_smm=', rho_smm)

alpha_smm= 0.45160303331885226 mu_smm= 9.842663921801112 sigma_smm= 0.07778608914972601 rho_smm= 0.6931743467625961


In [144]:
#Check convergence
results

     fun: array([[3.2697676e+14]])
     jac: array([9.42180777e+22, 3.14265213e+21, 2.57297487e+22, 2.77858888e+21])
 message: 'Converged (|x_n-x_(n-1)| ~= 0)'
    nfev: 47
     nit: 6
  status: 2
 success: True
       x: array([0.45160303, 9.84266392, 0.07778609, 0.69317435])

In [145]:
#Criterion function value
print("crit_val ", results.fun)

crit_val  [[3.2697676e+14]]


In [146]:
#Jacobian matrix
def jac_err(alpha, mu, sigma, rho, simple=True):
    '''
    This function computes the Jacobian matrix of partial derivatives of the R x 1 moment
    error vector e(x|theta) with respect to the K parameters theta_i in the K x 1 parameter vector
    theta. The resulting matrix is R x K Jacobian.
    '''
    jac_err_mat = np.zeros((6, 4))
    h_alpha = 1e-4 * alpha
    h_mu = 1e-4 * mu
    h_sigma = 1e-4 * sigma
    h_rho = 1e-4 * rho

    jac_err_mat[:, 0] = \
        ((err_vec(sim_fn(alpha + h_alpha, mu, sigma, rho), simple) -
          err_vec(sim_fn(alpha - h_alpha, mu, sigma, rho), simple)) / (2 * h_alpha)).flatten()
    jac_err_mat[:, 1] = \
        ((err_vec(sim_fn(alpha, mu + h_mu, sigma, rho), simple) -
          err_vec(sim_fn(alpha, mu - h_mu, sigma, rho), simple)) / (2 * h_mu)).flatten()  
    jac_err_mat[:, 2] = \
        ((err_vec(sim_fn(alpha, mu, sigma + h_sigma, rho), simple) -
          err_vec(sim_fn(alpha, mu, sigma - h_sigma, rho), simple)) / (2 * h_sigma)).flatten()
    jac_err_mat[:, 3] = \
        ((err_vec(sim_fn(alpha, mu, sigma, rho + h_rho), simple) -
          err_vec(sim_fn(alpha, mu, sigma, rho - h_rho), simple)) / (2 * h_rho)).flatten()


    return jac_err_mat

In [147]:
#Std error calculation
S = 1000
d_err = jac_err(alpha_smm, mu_smm, sigma_smm, rho_smm, False)
print("d_err", d_err)
print("Weight matrix", W_hat)
SigHat = (1 / S) * lin.inv(d_err.T @ W_hat @ d_err)
print("SigHat", SigHat)
print('Std. err. alpha_hat=', np.sqrt(SigHat[0, 0]))
print('Std. err. mu_hat=', np.sqrt(SigHat[1, 1]))
print('Std. err. sigma_hat=', np.sqrt(SigHat[2, 2]))
print('Std. err. rho_hat=', np.sqrt(SigHat[3, 3]))

d_err [[ 5.70333772e+01  3.45841056e+00  6.66222564e-01  9.28494884e-02]
 [ 7.30638528e+01  3.90674354e+00  7.52588699e-01  1.04886084e-01]
 [-1.67767888e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.95731440e+01  3.81090210e+00  2.57049977e+01  3.38353850e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.60165048e-12]
 [ 1.20264024e+01  6.00522545e-01  3.40280702e-01  9.39001450e-02]]
Weight matrix [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
SigHat [[ 2.80987808e-05 -4.96560619e-04  1.01839224e-04 -7.92174715e-04]
 [-4.96560619e-04  8.81656494e-03 -1.66018870e-03  1.28906279e-02]
 [ 1.01839224e-04 -1.66018870e-03  7.85617573e-03 -5.98890491e-02]
 [-7.92174715e-04  1.28906279e-02 -5.98890491e-02  4.56696645e-01]]
Std. err. alpha_hat= 0.005300828311392224
Std. err. mu_hat= 0.0938965650935572
Std. err. sigma_hat= 0.08863507054481588
Std. err. rho_hat= 0.675793344678198


# b) Perform the estimation using the two-step estimator for the optimal weighting matrix W2step.

In [148]:
#Define error matrix
def err_mat_fn(sim_vals, alpha, mu, sigma, rho, simple=True):
    c_sim, k_sim, w_sim, r_sim, y_sim, c_sh_sim, c_lag_sim = sim_vals
    R = 6
    S = 1000
    err_mat = np.zeros((R, S))
    m1_dta, m2_dta, m3_dta, m4_dta, m5_dta, m6_dta = moments_fn(c, k, w, r, y, c_sh, c_lag)
    m1_mod, m2_mod, m3_mod, m4_mod, m5_mod, m6_mod = moments_fn(c_sim, k_sim, w_sim, r_sim, y_sim, c_sh_sim, c_lag_sim)
    if simple:
        err_mat[0, :] = m1_mod - m1_dta
        err_mat[1, :] = m2_mod - m2_dta
        err_mat[2, :] = m3_mod - m3_dta
        err_mat[3, :] = m4_mod - m4_dta
        err_mat[4, :] = m5_mod - m5_dta
        err_mat[5, :] = m6_mod - m6_dta


    else:
        err_mat[0, :] = (m1_mod - m1_dta) / m1_dta
        err_mat[1, :] = (m2_mod - m2_dta) / m2_dta
        err_mat[2, :] = (m3_mod - m3_dta) / m3_dta
        err_mat[3, :] = (m4_mod - m4_dta) / m4_dta
        err_mat[4, :] = (m5_mod - m5_dta) / m5_dta
        err_mat[5, :] = (m6_mod - m6_dta) / m6_dta
    
    return err_mat

In [154]:
#Compute W_hat_2s
S=1000
c_sim_2s, k_sim_2s, w_sim_2s, r_sim_2s, y_sim_2s, c_sh_sim_2s, c_lag_sim_2s = sim_fn(alpha_smm, mu_smm, sigma_smm, rho_smm)
c_sim_2s, k_sim_2s, w_sim_2s, r_sim_2s, y_sim_2s, c_sh_sim_2s, c_lag_sim_2s = sim_vals_2s
err_mat_2s = err_mat_fn(sim_vals_2s, alpha_smm, mu_smm, sigma_smm, rho_smm, True)
print('err_mat', err_mat)
VCV2 = (1 / S) * (err_mat_2s @ err_mat_2s.T)
print(VCV2)
W_hat_2s = lin.inv(VCV2)
print(W_hat_2s)

[[ 7.12046395e+13  6.54047764e+13 -3.10664991e+05  1.18043961e+14
   9.36837736e-10 -5.27998566e+06]
 [ 6.54047764e+13  6.00773322e+13 -2.85360258e+05  1.08428874e+14
   8.60529077e-10 -4.84991265e+06]
 [-3.10664991e+05 -2.85360258e+05  1.35542765e-03 -5.15024393e+05
  -4.08741185e-18  2.30365144e-02]
 [ 1.18043961e+14  1.08428874e+14 -5.15024393e+05  1.95694788e+14
   1.55310157e-09 -8.75322771e+06]
 [ 9.36837736e-10  8.60529077e-10 -4.08741185e-18  1.55310157e-09
   1.23259516e-32 -6.94686448e-17]
 [-5.27998566e+06 -4.84991265e+06  2.30365144e-02 -8.75322771e+06
  -6.94686448e-17  3.91522923e-01]]
[[ 5.49464406e+00 -2.22041109e+00 -1.97733215e+08 -7.64143784e+00
   6.47855635e+23  2.34043420e+06]
 [-2.18400221e+00  1.36010109e+00  4.03518335e+08  3.20474968e+00
  -3.40744796e+23 -2.51580681e+07]
 [-1.26458629e+08  3.76026175e+08  2.88516230e+17  3.14286829e+06
   8.76111840e+31  1.59205631e+15]
 [-7.20491015e+00  3.07898088e+00  8.65557855e+07  5.08067995e+00
  -2.17107612e+23  1.095

In [151]:
#print(c_sim_2s, c_lag_sim_2s)

In [159]:
#Estimate by SMM
#Initial parameters
alpha_init = alpha_smm
beta = 0.99
mu_init = mu_smm
sigma_init = sigma_smm
rho_init = rho_smm
params_init_2s = np.array([alpha_init, mu_init, sigma_init, rho_init])
sim_init_2s = sim_fn(alpha_init, mu_init, sigma_init, rho_init)

smm_args_2s = (sim_init_2s, W_hat_2s)

results_2s = opt.minimize(criterion, params_init_2s, args=(smm_args_2s), method='L-BFGS-B', bounds=((0.01-(1e-10), 0.99+(1e-10)), (5-(1e-10), 14+(1e-10)), (0.01-(1e-10), 1.1+(1e-10)), (-0.99-(1e-10), 0.99+(1e-10))))

#results = opt.minimize(criterion, params_init, args=(smm_args), method='L-BFGS-B', bounds=((1e-10, 1-(1e-10)), (5+(1e-10), 14-(1e-10)), (1e-10, 1.1-(1e-10)), (-1+(1e-10), 1-(1e-10))))
alpha_smm_2s, mu_smm_2s, sigma_smm_2s, rho_smm_2s = results_2s.x
print('alpha_smm_2s=', alpha_smm_2s, 'mu_smm_2s=', mu_smm_2s, 'sigma_smm_2s=', sigma_smm_2s, 'rho_smm_2s=', rho_smm_2s)

alpha_smm_2s= 0.45160303331885226 mu_smm_2s= 9.842663921801112 sigma_smm_2s= 0.07778608914972601 rho_smm_2s= 0.6931743467625961


In [160]:
#Check convergence
results_2s

      fun: array([[0.93198243]])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([1.98044013e+23, 1.01083534e+23, 2.70343074e+22, 2.75978016e+22])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 105
      nit: 0
   status: 2
  success: False
        x: array([0.45160303, 9.84266392, 0.07778609, 0.69317435])

In [161]:
#Std error calculation
S = 1000
d_err_2s = jac_err(alpha_smm_2s, mu_smm_2s, sigma_smm_2s, rho_smm_2s, False)
print("d_err_2s", d_err_2s)
print("Weight matrix 2 Step", W_hat_2s)
SigHat_2s = (1 / S) * lin.inv(d_err_2s.T @ W_hat_2s @ d_err_2s)
print("SigHat_2s", SigHat_2s)
print('Std. err. alpha_hat_2s=', np.sqrt(SigHat_2s[0, 0]))
print('Std. err. mu_hat_2s=', np.sqrt(SigHat_2s[1, 1]))
print('Std. err. sigma_hat_2s=', np.sqrt(SigHat_2s[2, 2]))
print('Std. err. rho_hat_2s=', np.sqrt(SigHat_2s[3, 3]))

d_err_2s [[ 5.70333772e+01  3.45841056e+00  6.66222564e-01  9.28494884e-02]
 [ 7.30638528e+01  3.90674354e+00  7.52588699e-01  1.04886084e-01]
 [-1.67767888e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.95731440e+01  3.81090210e+00  2.57049977e+01  3.38353850e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.60165048e-12]
 [ 1.20264024e+01  6.00522545e-01  3.40280702e-01  9.39001450e-02]]
Weight matrix 2 Step [[ 5.49464406e+00 -2.22041109e+00 -1.97733215e+08 -7.64143784e+00
   6.47855635e+23  2.34043420e+06]
 [-2.18400221e+00  1.36010109e+00  4.03518335e+08  3.20474968e+00
  -3.40744796e+23 -2.51580681e+07]
 [-1.26458629e+08  3.76026175e+08  2.88516230e+17  3.14286829e+06
   8.76111840e+31  1.59205631e+15]
 [-7.20491015e+00  3.07898088e+00  8.65557855e+07  5.08067995e+00
  -2.17107612e+23  1.09500597e+07]
 [ 6.08528747e+23 -3.29145470e+23  8.06226677e+31 -2.13885011e+23
   1.82825472e+46 -2.15233731e+30]
 [ 1.37932191e+06 -2.47851864e+07  1.59628413e+15  1.21007467e

C:\Users\joset\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in sqrt
  
C:\Users\joset\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':
C:\Users\joset\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in sqrt
  # Remove the CWD from sys.path while we load stuff.
C:\Users\joset\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in sqrt
  # This is added back by InteractiveShellApp.init_path()
